# Heatmap of token probabilities

This notebook offers a simple script that can highlight "unexpected" token within a prompt. The highlighting is based on the difference between predicted token probability and actual token probability.

Example:

Consider the prompt 'print("Hello}World")'.
After processing 'print("Hello',the token '}' would get very low probability of being the next token. That's why the following script would highligt it. 

In [1]:
from transformers import GemmaTokenizer, AutoModelForCausalLM
import torch

In [2]:
gpu = torch.device('cuda:0')

In [3]:
model_id = "google/codegemma-1.1-7b-it"
tokenizer = GemmaTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map=gpu, torch_dtype=torch.float16)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu_pytorch_tanh`, edit the `model.config` to set `hidden_activation=gelu_pytorch_tanh`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
def escape_from_hex(r, g, b, text):
    return f"\x1B[38;2;{r};{g};{b}m{text}\x1B[0m"

In [5]:
print(escape_from_hex(255, 240, 0, "test"), escape_from_hex(255, 0, 0, "test"))

test test


In [9]:
prompt = """
You are debugging some code. Can you help me identify the issues in this piece of code? Can you give me for every line a probability if this line contains a bug:
def sumElementsOfList(in_list):
    sum = 0
    for i in list:
        sum -= i
    return sum
"""
#with open("convex_hull.py") as f:
#    prompt = f.read()

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#print("num_tokens", inputs["input_ids"].shape)
with torch.no_grad():
    logits = model(**inputs).logits[0]

differences = []

prompt_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
result = []
last_dif = 0
for j, i in enumerate(logits):
    probabilities = torch.softmax(i, 0)
    actual_token = torch.zeros([1], dtype=torch.int32)
    if j + 1 < len(inputs["input_ids"][0]):
        actual_token = inputs["input_ids"][0][j+1]
    act_prob = probabilities[actual_token]
    pred_prob = torch.max(probabilities)

    difference =  pred_prob.item() - act_prob.item()
    differences.append(difference)

    token = prompt_tokens[j]
    
    #prompt_tokens[j] = 
    #.replace("▁", " ")
    threshold = 0.2
    if last_dif <= threshold:
        token = token.replace("▁", " ")
    result.append(escape_from_hex(round(last_dif * 255), 0, 0, token))
    last_dif = difference
    if difference > threshold:
        pass
        #result.append(escape_from_hex(0, 255, 0, tokenizer.convert_ids_to_tokens([torch.argmax(probabilities)])[0]))
        #print(f"{difference}, {tokenizer.convert_ids_to_tokens([actual_token])}, {tokenizer.convert_ids_to_tokens([torch.argmax(probabilities)])}")

del inputs
del logits
del difference
torch.cuda.empty_cache()
# Print the prompt with colored tokens
colored_prompt = ''.join(result)
print(colored_prompt)

num_tokens torch.Size([1, 71])
<bos>
You are debugging▁some code.▁Can you help me▁identify the▁issues in▁this▁piece of code?▁Can you▁give me▁for▁every▁line a▁probability▁if▁this line▁contains▁a bug:
def▁sumElementsOfList(in_list):
    sum = 0
    for i in▁list:
        sum▁-=▁i
    return sum

